# Import

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid, train_test_split

In [2]:
# Read MIMICs CSV file
mimic_df = pd.read_csv("CSV\\exports\\final\\mimic_mean_median_min_max_final.csv")

# Read eICUs CSV file
eicu_df = pd.read_csv("CSV\\exports\\final\\eicu_mean_median_min_max_final.csv")

In [3]:
day = 10

# Filter icu stay less than 10 days
mimic_df = mimic_df[mimic_df['los'] < day]

# Filter icu stay less than 10 days
eicu_df = eicu_df[eicu_df['los'] < day]

In [4]:
"""
I'm gonna concat and split the mimic and icu
at this point. I must create the same columns
from the tranformation of categorical data.
"""
row_count = mimic_df.shape[0]
print(f"Row count: {row_count}")

# Concat dataframes
df_combined = pd.concat([mimic_df, eicu_df], ignore_index=True)

# Find all categorical columns in mimic
categorical_columns = df_combined.select_dtypes(include=['object', 'category']).columns.tolist()

# Apply one-hot encoding to all categorical columns
df_encoded = pd.get_dummies(df_combined, columns=categorical_columns)

# Split the concatenate dataframe
mimic_df = df_encoded.iloc[:row_count, :]
eicu_df = df_encoded.iloc[row_count:, :]

Row count: 51040


# Split Training - Validation - Test Set (Phase 02)

In [6]:
total_test_val_perc = 0.2
split_between_test_val_perc = 0.5

# Group data by subject_id and hadm_id
grouped_df = mimic_df.groupby(['subject_id', 'hadm_id'])

# Get a new dataframe with one row per patient (subject_id, hadm_id) pair
patient_df = grouped_df['hospital_expire_flag'].first().reset_index()

# Split the patient_df into training (80%), validation (10%), and test (10%) while keeping the ratio of hospital_expired_flag
train, temp = train_test_split(patient_df, test_size=total_test_val_perc, stratify=patient_df['hospital_expire_flag'], random_state=42)
val, test = train_test_split(temp, test_size=split_between_test_val_perc, stratify=temp['hospital_expire_flag'], random_state=42)

# Step 4: Merge back with the original df to get the rows for each patient in the splits
train_df = mimic_df.merge(train[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='inner')
val_df = mimic_df.merge(val[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='inner')
test_df = mimic_df.merge(test[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'], how='inner')

# Check the sizes of the splits
print(f'Training set size: {train_df.shape[0]}')
print(f'Validation set size: {val_df.shape[0]}')
print(f'Test set size: {test_df.shape[0]}')

Training set size: 40832
Validation set size: 5104
Test set size: 5104


# Split label from Train - Validation - Test Sets (Phase 04)

In [8]:
# External validation from eICU
X_external = eicu_df.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count', 'Time_Zone'])
y_external = eicu_df['los']

# Separate features and target for the training, validation, and test sets
X_train = train_df.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count', 'Time_Zone'])
y_train = train_df['los']

X_validate = val_df.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count', 'Time_Zone'])
y_validate = val_df['los']

X_test = test_df.drop(columns=['hospital_expire_flag', 'los', 'subject_id', 'hadm_id', 'row_count', 'Time_Zone'])
y_test = test_df['los']

# Load Train - Validation - Test & External Validation Sets

In [ ]:
# Define subfolder
subfolder = "o2_interpolation_impute"

# Load CSV files into corresponding variables
X_external = pd.read_csv(f"CSV/exports/impute/{subfolder}/X_external.csv")
y_external = pd.read_csv(f"CSV/exports/impute/{subfolder}/y_external.csv")
X_train = pd.read_csv(f"CSV/exports/impute/{subfolder}/X_train.csv")
y_train = pd.read_csv(f"CSV/exports/impute/{subfolder}/y_train.csv")
X_validate = pd.read_csv(f"CSV/exports/impute/{subfolder}/X_validate.csv")
y_validate = pd.read_csv(f"CSV/exports/impute/{subfolder}/y_validate.csv")
X_test = pd.read_csv(f"CSV/exports/impute/{subfolder}/X_test.csv")
y_test = pd.read_csv(f"CSV/exports/impute/{subfolder}/y_test.csv")

# GAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Custom Dataset class for PyTorch
def prepare_dataset(X):
    class CustomDataset(Dataset):
        def __init__(self, data):
            # Ensure data is converted to numeric type to avoid object type errors
            self.data = torch.tensor(data.astype(np.float32), dtype=torch.float32)

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            return self.data[idx]
    
    return CustomDataset(X)

# Define Generator model
class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        return self.main(x)

# Define Discriminator model
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

# GAN Imputation Function with Early Stopping
def impute_missing_values_with_gan(X, epochs=1000, batch_size=64, learning_rate=0.0002, patience=10):
    # Exclude columns 'age' and those starting with 'race_'
    columns_to_exclude = ['age'] + [col for col in X.columns if col.startswith('race_')]
    X_included = X.drop(columns=columns_to_exclude)

    # Preparing dataset
    X_missing = X_included.copy()
    mask = X_missing.isna()
    X_missing.fillna(0, inplace=True)
    dataset = prepare_dataset(X_missing.values)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    input_dim = X_included.shape[1]

    # Initialize models
    generator = Generator(input_dim)
    discriminator = Discriminator(input_dim)

    # Define optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

    # Define loss
    adversarial_loss = nn.BCELoss()

    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        for i, real_data in enumerate(dataloader):
            # Adversarial ground truths
            valid = torch.ones(real_data.size(0), 1)
            fake = torch.zeros(real_data.size(0), 1)

            # Configure input
            real_data = real_data

            # Train Generator
            optimizer_G.zero_grad()

            # Generate data with missing values replaced
            gen_data = generator(real_data)
            gen_data[mask.iloc[i * batch_size:(i + 1) * batch_size].values] = real_data[mask.iloc[i * batch_size:(i + 1) * batch_size].values]

            # Calculate generator loss
            g_loss = adversarial_loss(discriminator(gen_data), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()

            # Loss for real data
            real_loss = adversarial_loss(discriminator(real_data), valid)
            # Loss for fake data
            fake_loss = adversarial_loss(discriminator(gen_data.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2

            d_loss.backward()
            optimizer_D.step()

        print(f"Epoch {epoch + 1}/{epochs} - Generator Loss: {g_loss.item()}, Discriminator Loss: {d_loss.item()}")

        # Early stopping logic
        if g_loss.item() < best_loss:
            best_loss = g_loss.item()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1} - Generator Loss: {g_loss.item()}")
            break

    # Impute missing values using the trained generator
    X_imputed = X_missing.copy()
    X_imputed = torch.tensor(X_imputed.values.astype(np.float32), dtype=torch.float32)
    X_imputed = generator(X_imputed).detach().numpy()
    X_imputed[mask.values] = X_missing.values[mask.values]
    X_imputed_df = pd.DataFrame(X_imputed, columns=X_included.columns)

    # Apply mean imputation to excluded columns before combining with GAN-imputed data
    for col in columns_to_exclude:
        if X[col].isna().sum() > 0:
            X[col].fillna(X[col].mean(), inplace=True)

    # Combine imputed and excluded columns
    X_final = pd.concat([X_imputed_df, X[columns_to_exclude]], axis=1)
    X_final = X_final[X.columns]  # Reorder columns to match original dataframe

    return X_final

# Impute missing values for each dataset separately
X_external_imputed = impute_missing_values_with_gan(X_external)
X_train_imputed = impute_missing_values_with_gan(X_train)
X_validate_imputed = impute_missing_values_with_gan(X_validate)
X_test_imputed = impute_missing_values_with_gan(X_test)

Epoch 1/1000 - Generator Loss: 2.6760354042053223, Discriminator Loss: 0.2195051610469818
Epoch 2/1000 - Generator Loss: 2.2173211574554443, Discriminator Loss: 0.3854646682739258
Epoch 3/1000 - Generator Loss: 1.587363839149475, Discriminator Loss: 0.4536738395690918
Epoch 4/1000 - Generator Loss: 1.8108793497085571, Discriminator Loss: 0.5203495621681213
Epoch 5/1000 - Generator Loss: 1.1293985843658447, Discriminator Loss: 0.6584901809692383
Epoch 6/1000 - Generator Loss: 2.1951138973236084, Discriminator Loss: 0.24777957797050476
Epoch 7/1000 - Generator Loss: 1.4153923988342285, Discriminator Loss: 0.4504125714302063
Epoch 8/1000 - Generator Loss: 1.803475022315979, Discriminator Loss: 0.4983537197113037
Epoch 9/1000 - Generator Loss: 1.1499632596969604, Discriminator Loss: 0.4779139757156372
Epoch 10/1000 - Generator Loss: 1.6163921356201172, Discriminator Loss: 0.40899720788002014
Epoch 11/1000 - Generator Loss: 1.2909454107284546, Discriminator Loss: 0.4409679174423218
Epoch 12

In [11]:
# Path
save_path = 'CSV/exports/impute/o6_GAN/'

# Check if the directory exists, and if not, create it
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save external validation set from eICU
X_external_imputed.to_csv(save_path + 'X_external.csv', index=False)
y_external.to_csv(save_path + 'y_external.csv', index=False)

# Save training, validation, and test sets
X_train_imputed.to_csv(save_path + 'X_train.csv', index=False)
y_train.to_csv(save_path + 'y_train.csv', index=False)

X_validate_imputed.to_csv(save_path + 'X_validate.csv', index=False)
y_validate.to_csv(save_path + 'y_validate.csv', index=False)

X_test_imputed.to_csv(save_path + 'X_test.csv', index=False)
y_test.to_csv(save_path + 'y_test.csv', index=False)

In [17]:
X_validate_imputed.isna().sum()

age                                        0
Alanine_Aminotransferase_(ALT)_(Max)       0
Alanine_Aminotransferase_(ALT)_(Mean)      0
Alanine_Aminotransferase_(ALT)_(Median)    0
Alanine_Aminotransferase_(ALT)_(Min)       0
                                          ..
race_WHITE                                 0
race_WHITE - BRAZILIAN                     0
race_WHITE - EASTERN EUROPEAN              0
race_WHITE - OTHER EUROPEAN                0
race_WHITE - RUSSIAN                       0
Length: 344, dtype: int64

In [18]:
total_missing_values = X_external_imputed.isna().sum().sum()
print(total_missing_values)

17557760
